In [1]:
#To export data to Excel files
!pip install openpyxl


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Rahav Shema\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

# Load the dataset
file_path = './waze.csv'
dataset = pd.read_csv(file_path)

# Clean the dataset by dropping rows with missing 'label'
dataset_cleaned = dataset.dropna(subset=['label']).copy()

# and removing the 'sessions' column (stay with the 'total_sessions')
dataset_cleaned = dataset_cleaned.drop(columns=['sessions'])

# dropping rows with "n_days_after_onboarding" (tatal) under 2 years
dataset_cleaned = dataset_cleaned[dataset_cleaned['n_days_after_onboarding'] > 730]

# dropping rows with "drives" (last month) equals to 0
dataset_cleaned = dataset_cleaned[dataset_cleaned['drives'] != 0]

# dropping rows with "total_sessions" (total) under 50
dataset_cleaned = dataset_cleaned[dataset_cleaned['total_sessions'] > 50]

# dropping rows with "total_navigations_fav1/2" (total) equals to 0
dataset_cleaned = dataset_cleaned[dataset_cleaned['total_navigations_fav1'] != 10]
dataset_cleaned = dataset_cleaned[dataset_cleaned['total_navigations_fav2'] != 10]

# dropping rows with "driven_km_drives" (total) under 200
dataset_cleaned = dataset_cleaned[dataset_cleaned['driven_km_drives'] > 200]

# dropping rows with "duration_minutes_drives" (last month) under 2 hours
dataset_cleaned = dataset_cleaned[dataset_cleaned['duration_minutes_drives'] > 120]

# dropping rows with "activity_days" (last month) equals to 0
dataset_cleaned = dataset_cleaned[dataset_cleaned['activity_days'] != 0]

# dropping rows with "driving_days" (last month) equals to 0
dataset_cleaned = dataset_cleaned[dataset_cleaned['driving_days'] != 0]

# Get the number of records
print(f"Number of records: {len(dataset_cleaned)}")

# Export to CSV
output_file_path = './cleaned_waze_app_dataset.csv'
dataset_cleaned.to_csv(output_file_path, index=False)
print(f"The cleaned dataset has been exported to {output_file_path}")

pd.DataFrame(dataset_cleaned).head(5)

FileNotFoundError: [Errno 2] No such file or directory: './waze.csv'

In [ ]:
import networkx as nx

# Load the cleaned dataset
file_path = './cleaned_waze_app_dataset.csv'
dataset_cleaned = pd.read_csv(file_path)

# Split the dataset into iPhone and Android users
iphone_users = dataset_cleaned[dataset_cleaned['device'] == 'iPhone']
android_users = dataset_cleaned[dataset_cleaned['device'] == 'Android']
pd.DataFrame(iphone_users).head(5)
print(len(android_users))
print(len(iphone_users))


3335
5937


In [3]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
file_path = './cleaned_waze_app_dataset.csv'
dataset_cleaned = pd.read_csv(file_path)

# Take the first 500 rows of the dataset
dataset_sample = dataset_cleaned.head(2000)

# Split the sample into iPhone and Android users
iphone_users_sample = dataset_sample[dataset_sample['device'] == 'iPhone']
android_users_sample = dataset_sample[dataset_sample['device'] == 'Android']

# Define a function to create a graph
def create_graph(users, threshold_km):
    G = nx.Graph()
    for idx, row in users.iterrows():
        G.add_node(row['ID'], total_sessions=row['total_sessions'], activity_days=row['activity_days'],
                   driven_km_drives=row['driven_km_drives'], duration_minutes_drives=row['duration_minutes_drives'],
                   device=row['device'], label=row['label'])
    for i, row1 in users.iterrows():
        for j, row2 in users.iterrows():
            if i < j and abs(row1['driven_km_drives'] - row2['driven_km_drives']) < threshold_km:
                G.add_edge(row1['ID'], row2['ID'], weight=abs(row1['driven_km_drives'] - row2['driven_km_drives']))
    return G

# Create graphs for iPhone and Android users in the sample
threshold_km = 5
G_iphone_sample = create_graph(iphone_users_sample, threshold_km)
G_android_sample = create_graph(android_users_sample, threshold_km)

# Define a function to draw the graph
def draw_graph(G, title, node_color):
    plt.figure(figsize=(14, 10))  # Increase the figure size
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_size=100, node_color=node_color, font_size=10, font_weight='bold')  # Increase node size
    plt.title(title)
    plt.show()

# Draw the graph for iPhone users
draw_graph(G_iphone_sample, 'User Interaction Graph for iPhone Users (Sample)', 'skyblue')

# Draw the graph for Android users
draw_graph(G_android_sample, 'User Interaction Graph for Android Users (Sample)', 'lightgreen')

# Function to detect communities using the Girvan-Newman algorithm
def detect_communities(G):
    communities = nx.community.girvan_newman(G)
    top_level_communities = next(communities)
    sorted_communities = sorted(map(sorted, top_level_communities))
    return sorted_communities

# Detect communities in the iPhone users graph
iphone_communities = detect_communities(G_iphone_sample)
print("Communities in iPhone users graph:", iphone_communities)

# Detect communities in the Android users graph
android_communities = detect_communities(G_android_sample)
print("Communities in Android users graph:", android_communities)

# Function to draw communities
def draw_communities(G, communities, title):
    plt.figure(figsize=(14, 10))  # Increase the figure size
    pos = nx.spring_layout(G, seed=42)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(communities)))
    for community, color in zip(communities, colors):
        nx.draw_networkx_nodes(G, pos, nodelist=community, node_color=[color], label=community, node_size=100)  # Increase node size
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    plt.title(title)
    plt.show()

# Draw communities for iPhone users
draw_communities(G_iphone_sample, iphone_communities, 'Communities in iPhone Users Graph')

# Draw communities for Android users
draw_communities(G_android_sample, android_communities, 'Communities in Android Users Graph')



FileNotFoundError: [Errno 2] No such file or directory: './cleaned_waze_app_dataset.csv'